# Xband Bathy Analysis

## Initial Data Analysis Setup

### Imports

In [15]:
# Imports
import scipy.io as spio
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import netCDF4
import datetime
import pytz
import scipy.stats as stats
import scipy.optimize as spo
from sklearn.metrics import r2_score

#### Create Bathy File Directory List

In [16]:
# Initialize empty array
bathyFiles = []

# Import files from directory
directory = '2022-2021_Xband_Data'
for root, dirs, files in os.walk(directory):
    for file in files:
        if file.endswith('.RIOS.Bathyproc.mat'):
            bathyFiles.append(os.path.join(root, file))


# Sort chronologically
bathyFiles = sorted(bathyFiles)

#### Names Bathy Files Based on Date/Time and Creates Dictionary

In [17]:
# Initialize empty array
bathyFilesDateAndTime = []

# Name files based on date-time
for i in bathyFiles:
    bathyFile = i
    bathyFile = str(bathyFile[92:105])
    bathyFilesDateAndTime.append(bathyFile)

# Create key/value lists
bathyFilesKey = bathyFiles
bathyFilesValue = bathyFilesDateAndTime

# New dictionary from key/value pairing
bathyFilesDateAndTimeDict = dict(zip(bathyFilesKey, bathyFilesValue))

print(f'Length: {np.size(bathyFilesDateAndTime)}')

Length: 1988


#### Convert .mat Data Into Python Arrays

In [ ]:
# Initialize arrays
# beachStart = []
# eastingCenter = []
# iGrid = []
# iGridMean = []
# meanWaveAngle = []
# northingCenter = []
# pktpFilt = []
# swashStart = []
# swashStop = []
# tpCollect = []
# tpMean = []
xGrid = []
# xshoreStart = []
# xshoreStop = []
yGrid = []
depthMean = []
depthSmooth = []
# depthSmoothR = []
# f = []
# profX = []

# Add data from each available time to corresponding array
for i in bathyFiles:
    if os.stat(i).st_size != 0:
        currentFile = spio.loadmat(i)
        # beachStart.append(currentFile['Beach_start'])
        # eastingCenterList.append(currentFile['EastingCenter'])
        # iGrid.append(currentFile['IGrid'])
        # iGridMean.append(currentFile['IGridMean'])
        # meanWaveAngle.append(currentFile['MeanWaveAngle_geo'])
        # northingCenterList.append(currentFile['NorthingCenter'])
        # pktpFilt.append(currentFile['PkTp_filt'])
        # swashStart.append(currentFile['Swash_start'])
        # swashStop.append(currentFile['Swash_stop'])
        # tpCollect.append(currentFile['Tp_collect'])
        # tpMean.append(currentFile['Tp_mean'])
        xGrid.append(currentFile['XGrid'])
        # xshoreStart.append(currentFile['Xshore_start'])
        # xshoreStop.append(currentFile['Xshore_stop'])
        yGrid.append(currentFile['YGrid'])
        depthMean.append(np.asarray(currentFile['depthMean']))
        depthSmooth.append(np.asarray(currentFile['depthSmooth']))
        # depthSmoothR.append(currentFile['depthSmooth_R'])
        # f.append(currentFile['f'])
        # profX.append(currentFile['prof_x'])

#### Calculates Number of Nan Values Per Grid

In [ ]:
# Initialize empty arrays
depthSmoothNanKeyArray = []
depthSmoothNanValueArray = []

# Iterate through arrays to find nan count
key = 1
for i in depthSmooth:
    depthSmoothNanKeyArray.append(key)
    depthSmoothNanValueArray.append(np.count_nonzero(np.isnan(i)))
    key += 1

# Create dictionary from nan counts
depthSmoothNanCountDict = dict(zip(depthSmoothNanKeyArray, depthSmoothNanValueArray))

# Plot nan counts at each time period
plt.title('Nan values in dataset at each time period')
plt.scatter(depthSmoothNanKeyArray, depthSmoothNanValueArray, color = '#5CA892')
plt.xlabel('Date-Time')
plt.ylabel('Number of Nan Values')
plt.xticks(depthSmoothNanKeyArray, bathyFilesDateAndTime)
plt.locator_params(axis = 'both', nbins = 5)
plt.tick_params(axis = 'x', rotation = 45)
plt.grid()
plt.show()

#### Station Number and Import Libraries

In [ ]:
# Set station identification variables
stn = '243'
startdate = '12/13/2021'
enddate = '12/17/2022'
station = 'https://thredds.cdip.ucsd.edu/thredds/dodsC/cdip/archive/243p1/243p1_historic.nc?waveHs[0:1:82385],waveTa[0:1:82385],sstTime[0:1:82387]'

# Convert data to netCDF dataset
stationNC = netCDF4.Dataset(station)
print(stationNC.variables.keys())

# Create arrays of imported variables
waveHs = stationNC.variables['waveHs']
waveHs = np.asarray(waveHs[:]) * 0.3048 # units: m

waveTa = stationNC.variables['waveTa']
waveTa = np.asarray(waveTa[:]) # units: s

sstTime = stationNC.variables['sstTime']
sstTime = np.asarray(sstTime[:])

data = [waveHs, waveTa]

#### Comparing CDIP Data with XBand Data

In [ ]:
# Initialize empty array
unixTimestamp = []

# Iterate through files and convert unformatted date-time to formatted date-time
for i in bathyFilesDateAndTime:
    dateObj = datetime.datetime.strptime(i, '%Y%m%d-%H%M')
    timezone = pytz.timezone('America/New_York')
    localDate = timezone.localize(dateObj)
    utcDate = localDate.astimezone(pytz.utc)
    unixTimestamp.append(utcDate.timestamp())

# function to match closest values in an array to given value (used for time correspondence between data sources)
def FindNearest(array, value):
    array = np.asarray(array)
    match = (np.abs(array - value)).argmin()
    return array[match]

# Array of times matching between cdip and xband
cdipClosestTimes = []

# Iterate through xband times to find nearest cdip times and add to array
for i in unixTimestamp:
    nearestTime = FindNearest(sstTime, i)
    cdipClosestTimes.append(nearestTime)

# Initialize empty array
cdipClosestTimesIndex = []

# Find indexes of where times close to xband occur on cdip
sstTimeList = list(sstTime)
for i in cdipClosestTimes:
    x = sstTimeList.index(i)
    cdipClosestTimesIndex.append(x)

# Initialize empty arrays
waveHsClosestTimes = []
waveTaClosestTimes = []

# Find wave height and period values based on closest time index
for i in cdipClosestTimesIndex:
    waveHsClosestTimes.append(waveHs[i])
    waveTaClosestTimes.append(waveTa[i])

# Plots
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax1.plot(bathyFilesDateAndTime, depthSmoothNanValueArray, color = '#5CA892', label = 'Nan Value Count')
ax2.plot(bathyFilesDateAndTime, waveHsClosestTimes, color = '#C8A160', label = 'Wave Height (meters)')

ax1.set_xlabel('Date-Time')
ax1.set_ylabel('NaN Value Count')
ax1.xaxis.set_major_locator(plt.MaxNLocator(5))
ax1.tick_params(axis = 'y')
ax1.legend()

ax2.set_ylabel('Wave Height')
ax2.tick_params(axis = 'y')
ax2.legend()

fig.autofmt_xdate()
plt.title('NaN Counts Compared To Wave Height')
plt.grid()
plt.show()

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax1.plot(bathyFilesDateAndTime, depthSmoothNanValueArray, color = '#5CA892', label = 'Nan Value Count')
ax2.plot(bathyFilesDateAndTime, waveTaClosestTimes, color = '#E5DBC1', label = 'Wave Period (seconds)')

ax1.set_xlabel('Date-Time')
ax1.set_ylabel('NaN Value Count')
ax1.xaxis.set_major_locator(plt.MaxNLocator(5))
ax1.tick_params(axis = 'y')
ax1.legend()

ax2.set_ylabel('Wave Period')
ax2.tick_params(axis = 'y')
ax2.legend()

fig.autofmt_xdate()
plt.title('NaN Counts Compared To Wave Period')
plt.grid()
plt.show()

#### Create Heatmap of Bathymetry

In [ ]:
# Set up coordinate grid for data
xDistanceUTM = xGrid[0]
yDistanceUTM = yGrid[0]

radarXUTM = np.median(xDistanceUTM)
radarYUTM = np.median(yDistanceUTM)

xDistance = xDistanceUTM - radarXUTM
yDistance = yDistanceUTM - radarYUTM

xDistance = xDistance[0, :]
yDistance = yDistance[:, 0]

xDistance.sort()
yDistance.sort()

# n = 0

# for i in depthMean:
#     dataset = i
#     plt.title(bathyFilesDateAndTime[n])
#     cmapColorScheme = sns.dark_palette('#5CA892', as_cmap=True)
#     ax = sns.heatmap(dataset, square = 'True', xticklabels = xDistance, yticklabels = yDistance, cmap = cmapColorScheme)
#     plt.locator_params(axis = 'x', nbins = 11)
#     plt.locator_params(axis = 'y', nbins = 16)
#     ax.collections[0].colorbar.set_label('Meters Deep')
#     ax.add_patch(patches.Rectangle((57, 146), 20, 63, fill = False, angle = 295))
#     plt.xlabel('Horizontal Meters from Radar')
#     plt.ylabel('Vertical Meters from Radar')
#     plt.savefig('/BathyHeatmaps' + bathyFilesDateAndTime[n] + '.png', bbox_inches = 'tight')
#     plt.show()
#     n += 1

#### Calculates Number of Total Points, Points Per Grid, Points Per Row/Column, Radar Resolution

In [ ]:
cols = len(depthSmooth[0][0])
rows = len(depthSmooth[0])

print('Columns, Rows')
print(cols, rows)

pointsPerGrid = rows * cols
print('Points per grid: ', pointsPerGrid)

totalPoints = pointsPerGrid * len(depthSmooth)
print('Total number of points: ', totalPoints)

xDims = 1000 / cols
yDims = 1600 / rows
print(f'Dimensions: {xDims}m x {yDims}m')
print(f'Square meters per point: {xDims * yDims} m^2')
metersSquaredPerPoint = xDims * yDims # units: m^2

#### Change in Depth Heatmaps

In [ ]:
# Initialize empty array
depthChangeDatasets = []

# Iterate through datasets and calculate sand change
i = 0
while i < (len(depthMean) - 2):
    file0 = depthMean[i]
    file1 = depthMean[i + 1]
    diffArray = file1 - file0
    i += 1
    depthChangeDatasets.append(diffArray)
    
    # cmapColorScheme = sns.diverging_palette(162.63, 37.5, center = 'light', as_cmap = True)
    # ax = sns.heatmap(diffArray, center = 0, xticklabels = xDistance, yticklabels = yDistance, cmap = cmapColorScheme)
    # plt.locator_params(axis = 'x', nbins = 11)
    # plt.locator_params(axis = 'y', nbins = 16)
    # ax.collections[0].colorbar.set_label('Meters (difference)')
    # ax.add_patch(patches.Rectangle((57, 146), 20, 63, fill = False, angle = 295))
    # ax.set_xlabel('Horizontal Meters from Radar')
    # ax.set_ylabel('Vertical Meters from Radar')
    # plt.title('Depth Change ' + bathyFilesDateAndTime[i] + '-' + bathyFilesDateAndTime[i + 1])
    # plt.savefig('/DeltaHeatmaps' + bathyFilesDateAndTime[i] + '-' + bathyFilesDateAndTime[i + 1] + '.png', bbox_inches = 'tight')
    # plt.show()

#### Volume Moved Sand Calculation

In [ ]:
# Initialize empty arrays
totalSandMoved = []
changeWorkingDatasetPositive = []

# Iterate through datasets to find volume change
for i in depthChangeDatasets:
    changeWorkingDatasetPositive = []
    workingDataset = np.array(i)[np.logical_not(np.isnan(i))].tolist()
    workingTotalSandMoved = 0

    for w in workingDataset:
        changeWorkingDatasetPositive.append(abs(w) * metersSquaredPerPoint)
        workingTotalSandMoved += (abs(w) * metersSquaredPerPoint)

    totalSandMoved.append(workingTotalSandMoved)

print(totalSandMoved)

#### Wave Energy Calculation

In [ ]:
# Initialize empty array
waveEnergy = []

# Iterate through datasets to find wave energy
i = 0
while i < len(cdipClosestTimes):
    workingWaveEnergy = (0.125) * (1027) * (9.81) * (waveHs[i] ** 2) # Units: Joules/m^2
    # Source (wave energy density): https://x-engineer.org/wave-energy/
    # Source (ocean saltwater density in kg/m^3): http://hyperphysics.phy-astr.gsu.edu/hbase/Chemical/seawater.html
    waveEnergy.append(workingWaveEnergy)
    i += 1

print(waveEnergy)

#### Show Distribution (Q1, Median, Q3) of Variables

In [ ]:
# Find distribution for each dataset
waveHsQ1, waveHsMedian, waveHsQ3 = np.quantile(waveHsClosestTimes, [0.25, 0.5, 0.75])
waveTaQ1, waveTaMedian, waveTaQ3 = np.quantile(waveTaClosestTimes, [0.25, 0.5, 0.75])
sandMovedQ1, sandMovedMedian, sandMovedQ3 = np.quantile(totalSandMoved, [0.25, 0.5, 0.75])
waveEnergyQ1, waveEnergyMedian, waveEnergyQ3 = np.quantile(waveEnergy, [0.25, 0.5, 0.75])

# Print out data stats
print('Wave height stats:')
print(f'Length: {len(waveHsClosestTimes)}')
print(f'Q1: {waveHsQ1}, Median: {waveHsMedian}, Q3: {waveHsQ3}')
print()

print('Wave period stats:')
print(f'Length: {len(waveTaClosestTimes)}')
print(f'Q1: {waveTaQ1}, Median: {waveTaMedian}, Q3: {waveTaQ3}')
print()

print('Sand moved stats:')
print(f'Length: {len(totalSandMoved)}')
print(f'Q1: {sandMovedQ1}, Median: {sandMovedMedian}, Q3: {sandMovedQ3}')
print()

print('Wave energy stats:')
print(f'Length: {len(waveEnergy)}')
print(f'Q1: {waveEnergyQ1}, Median: {waveEnergyMedian}, Q3: {waveEnergyQ3}')
print()

#### Analyze Percent Change Metrics

In [ ]:
levelOneWaveHeightTimesTotalSandMoved = []
levelTwoWaveHeightTimesTotalSandMoved = []
levelThreeWaveHeightTimesTotalSandMoved = []
levelFourWaveHeightTimesTotalSandMoved = []

levelOneWavePeriodTimesTotalSandMoved = []
levelTwoWavePeriodTimesTotalSandMoved = []
levelThreeWavePeriodTimesTotalSandMoved = []
levelFourWavePeriodTimesTotalSandMoved = []

levelOneWaveEnergyTimesTotalSandMoved = []
levelTwoWaveEnergyTimesTotalSandMoved = []
levelThreeWaveEnergyTimesTotalSandMoved = []
levelFourWaveEnergyTimesTotalSandMoved = []

i = 1
while i < (len(waveEnergy) - 2):
    if waveHsClosestTimes[i] <= waveHsQ1:
        levelOneWaveHeightTimesTotalSandMoved.append(totalSandMoved[i])
    elif waveHsClosestTimes[i] > waveHsQ1 and waveHsClosestTimes[i] <= waveHsMedian:
        levelTwoWaveHeightTimesTotalSandMoved.append(totalSandMoved[i])
    elif waveHsClosestTimes[i] > waveHsMedian and waveHsClosestTimes[i] <= waveHsQ3:
        levelThreeWaveHeightTimesTotalSandMoved.append(totalSandMoved[i])
    elif waveHsClosestTimes[i] > waveHsQ3:
        levelFourWaveHeightTimesTotalSandMoved.append(totalSandMoved[i])

    if waveTaClosestTimes[i] <= waveTaQ1:
        levelOneWavePeriodTimesTotalSandMoved.append(totalSandMoved[i])
    elif waveTaClosestTimes[i] > waveTaQ1 and waveTaClosestTimes[i] <= waveTaMedian:
        levelTwoWavePeriodTimesTotalSandMoved.append(totalSandMoved[i])
    elif waveTaClosestTimes[i] > waveTaMedian and waveTaClosestTimes[i] <= waveTaQ3:
        levelThreeWavePeriodTimesTotalSandMoved.append(totalSandMoved[i])
    elif waveTaClosestTimes[i] > waveTaQ3:
        levelFourWavePeriodTimesTotalSandMoved.append(totalSandMoved[i])

    if waveEnergy[i] <= waveEnergyQ1:
        levelOneWaveEnergyTimesTotalSandMoved.append(totalSandMoved[i])
    elif waveEnergy[i] > waveEnergyQ1 and waveEnergy[i] <= waveEnergyMedian:
        levelTwoWaveEnergyTimesTotalSandMoved.append(totalSandMoved[i])
    elif waveEnergy[i] > waveEnergyMedian and waveEnergy[i] <= waveEnergyQ3:
        levelThreeWaveEnergyTimesTotalSandMoved.append(totalSandMoved[i])
    elif waveEnergy[i] > waveEnergyQ3:
        levelFourWaveEnergyTimesTotalSandMoved.append(totalSandMoved[i])
    
    i += 1

levelOneHeightData = np.array(levelOneWaveHeightTimesTotalSandMoved)
levelTwoHeightData = np.array(levelTwoWaveHeightTimesTotalSandMoved)
levelThreeHeightData = np.array(levelThreeWaveHeightTimesTotalSandMoved)
levelFourHeightData = np.array(levelFourWaveHeightTimesTotalSandMoved)

levelOnePeriodData = np.array(levelOneWavePeriodTimesTotalSandMoved)
levelTwoPeriodData = np.array(levelTwoWavePeriodTimesTotalSandMoved)
levelThreePeriodData = np.array(levelThreeWavePeriodTimesTotalSandMoved)
levelFourPeriodData = np.array(levelFourWavePeriodTimesTotalSandMoved)

levelOneEnergyData = np.array(levelOneWaveEnergyTimesTotalSandMoved)
levelTwoEnergyData = np.array(levelTwoWaveEnergyTimesTotalSandMoved)
levelThreeEnergyData = np.array(levelThreeWaveEnergyTimesTotalSandMoved)
levelFourEnergyData = np.array(levelFourWaveEnergyTimesTotalSandMoved)

totalSandMovedHeightData = [levelOneHeightData, levelTwoHeightData, levelThreeHeightData, levelFourHeightData]
totalSandMovedPeriodData = [levelOnePeriodData, levelTwoPeriodData, levelThreePeriodData, levelFourPeriodData]
totalSandMovedEnergyData = [levelOneEnergyData, levelTwoEnergyData, levelThreeEnergyData, levelFourEnergyData]

labelsHeight = [f'{round(waveHsQ1, 3)}- m', f'{round(waveHsQ1, 3)} - {round(waveHsMedian, 3)} m', f'{round(waveHsMedian, 3)} - {round(waveHsQ3, 3)} m', f'{round(waveHsQ3, 3)}+ m']
labelsPeriod = [f'{round(waveTaQ1, 3)}- m', f'{round(waveTaQ1, 3)} - {round(waveTaMedian, 3)} m', f'{round(waveTaMedian, 3)} - {round(waveTaQ3, 3)} m', f'{round(waveTaQ3, 3)}+ m']
labelsEnergy = [f'{round(waveEnergyQ1, 3)}- m', f'{round(waveEnergyQ1, 3)} - {round(waveEnergyMedian, 3)} m', f'{round(waveEnergyMedian, 3)} - {round(waveEnergyQ3, 3)} m', f'{round(waveEnergyQ3, 3)}+ m']

colors = ['#5CA892', '#C8A160', '#E5DBC1', '#0D3639', '#FFFFFF']

bp1 = plt.boxplot(totalSandMovedHeightData, vert = True, patch_artist =True, labels = labelsHeight, showfliers = False, notch = True)
plt.ylabel('Change In Sand (m^3)')
plt.xticks(rotation = 10)
plt.title('Sand Moved Across Small and Large Waves')

for patch, color in zip(bp1['boxes'], colors):
    patch.set_facecolor(color)
for medians in bp1['medians']:
    medians.set(color = '#000000')
plt.show()

bp2 = plt.boxplot(totalSandMovedPeriodData, vert = True, patch_artist = True, labels = labelsPeriod, showfliers = False, notch = True)
plt.ylabel('Change In Sand (m^3)')
plt.xticks(rotation = 10)
plt.title('Sand Moved Across Short and Long Period Waves')

for patch, color in zip(bp2['boxes'], colors):
    patch.set_facecolor(color)
for medians in bp2['medians']:
    medians.set(color = '#000000')
plt.show()

bp3 = plt.boxplot(totalSandMovedEnergyData, vert = True, patch_artist = True, labels = labelsEnergy, showfliers = False, notch = True)
plt.ylabel('Change In Sand (m^3)')
plt.xticks(rotation = 10)
plt.title('Sand Moved Across High and Low Energy Waves')

for patch, color in zip(bp3['boxes'], colors):
    patch.set_facecolor(color)
for medians in bp3['medians']:
    medians.set(color = '#000000')
plt.show()

bp4 = plt.boxplot(totalSandMoved, showfliers = False, vert=True, patch_artist=True, notch=True)
plt.title('Total Sand Moved (m^3)')

for patch, color in zip(bp4['boxes'], colors):
    patch.set_facecolor(color)
for medians in bp4['medians']:
    medians.set(color = '#000000')
plt.show()

#### Apply Statistical Tests to Data

In [ ]:
statHeight, pValueHeight = stats.f_oneway(levelOneHeightData, levelTwoHeightData, levelThreeHeightData, levelFourHeightData)
statPeriod, pValuePeriod = stats.f_oneway(levelOnePeriodData, levelTwoPeriodData, levelThreePeriodData, levelFourPeriodData)
statEnergy, pValueEnergy = stats.f_oneway(levelOneEnergyData, levelTwoEnergyData, levelThreeEnergyData, levelFourEnergyData)

print('P-Values: Height, Period, Energy')
print(pValueHeight)
print(pValuePeriod)
print(pValueEnergy)
print()

print('Stat values: Height, Period, Energy')
print(statHeight)
print(statPeriod)
print(statEnergy)
print()

print('Sample size: Height, Period, Energy')
print(len(waveHsClosestTimes))
print(len(waveTaClosestTimes))
print(len(waveEnergy))

#### Plot Relationships Between Sand Moved And Wave Variables

In [ ]:
# Plots
plt.scatter(waveHsClosestTimes[:-2], totalSandMoved, color = '#5CA892')
plt.xlabel('Wave Height (m)')
plt.ylabel('Sand Moved (m^3)')
plt.title('Total Sand Moved versus Wave Height')
plt.grid()
plt.show()

plt.scatter(waveTaClosestTimes[:-2], totalSandMoved, color = '#5CA892')
plt.xlabel('Wave Period (s)')
plt.ylabel('Sand Moved (m^3)')
plt.title('Total Sand Moved versus Wave Period')
plt.grid()
plt.show()

plt.scatter(waveEnergy[:-2], totalSandMoved, color = '#5CA892')
plt.xlabel('Wave Energy Density (J/m^3)')
plt.ylabel('Sand Moved (m^3)')
plt.title('Total Sand Moved versus Wave Energy Density')
plt.grid()
plt.show()

In [ ]:
# Generate equation formats
def Linear(x, a, b):
    return (np.array(x) * a) + b

def Inverse(x, k):
    return k / np.array(x)

# Find best fit parameters
coefficientsHs, covarianceHs = spo.curve_fit(Linear, waveHsClosestTimes[:-2], totalSandMoved)
aHs, bHs = coefficientsHs

coefficientsEnergy, covarianceEnergy = spo.curve_fit(Inverse, waveEnergy[:-2], totalSandMoved, p0 = [1 / 0.00000002])
aEnergy = coefficientsEnergy

# Plots
plt.scatter(waveHsClosestTimes[:-2], totalSandMoved, color = '#5CA892')
plt.plot(waveHsClosestTimes[:-2], Linear(waveHsClosestTimes[:-2], aHs, bHs), color = '#0D3639')
plt.xlabel('Wave Height (m)')
plt.ylabel('Sand Moved (m^3)')
plt.title('Total Sand Moved versus Wave Height')
plt.grid()
plt.show()

plt.scatter(waveEnergy[:-2], totalSandMoved, color = '#5CA892')
plt.plot(waveEnergy[:-2], Inverse(waveEnergy[:-2], aEnergy), color = '#0D3639')
plt.xlabel('Wave Energy Density (J/m^3)')
plt.ylabel('Sand Moved (m^3)')
plt.title('Total Sand Moved versus Wave Energy Density')
plt.grid()
plt.show()

print(aEnergy)

#### Calculate R Values

In [ ]:
waveHeightResults = Linear(waveHsClosestTimes[:-2], aHs, bHs)
waveEnergyResults = Inverse(waveEnergy[:-2], aEnergy)

r2WaveHeight = r2_score(totalSandMoved, waveHeightResults)
r2WaveEnergy = r2_score(totalSandMoved, waveEnergyResults)

print('R^2 Values:')
print(f'Wave height: {np.round(r2WaveHeight, 3)}')
print(f'Wave energy density: {np.round(r2WaveEnergy, 3)}')

In [ ]:
print(coefficientsHs)
print(coefficientsEnergy)